In [123]:
! pip install beautifulsoup4

In [124]:
! pip install selenium

In [125]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

import time

In [126]:
# We open Chrome browser in incognito mode
service = ChromeService(executable_path=ChromeDriverManager().install())

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome(service=service, chrome_options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Mikel\.wdm\drivers\chromedriver\win32\99.0.4844.51]
<ipython-input-126-d681e9ce9338>:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=service, chrome_options=chrome_options)


In [127]:
# Link to the desired page to be scraped
driver.get('https://foxhugh.com/interview-questions/list-of-interview-questions/');

# Save all the content in the "soup" variable
content = driver.page_source
soup = BeautifulSoup(content)

#List to store de questions
allQuestions = [] 

#The content we want is in a "ol" type layer 
new = soup.ol

#Get the content
hijos=new.findAll('li')

#Iterate the content 1 by 1
for child in hijos:
    print(child.string)
    allQuestions.append(child.string)

A natural disaster occurs in Manhattan. What impact does this have on markets?
Add up all the numbers between 0-100.
Are there any particular companies that interest you?
Are there aspects to your job that are repetitious?
Are you a Coronation Street or EastEnders fan?
Are you a goal-oriented person?
Are you a leader or a follower?
Are you a lone wolf?
Are you a team player?
Are you better with data or people?
Are you comfortable with a lack of deadlines?
Are you comfortable with strict deadlines?
Are you currently taking or enrolled to take any job related educational courses?
Are you currently under any employment contract obligation form current or previous employers?
Are you good at accepting delegation?
Are you good at delegating tasks?
Are you hoping for advancement within the company?
Are you looking for a permanent or temporary position at the company?
Are you the type of student for whom conducting independent research has been a positive experience?
Are you willing to make sa

In [128]:
# Loop to delete duplicated and None questions (in case that there are)
questions = []
 
for element in allQuestions:
    if element != None :
        if element not in questions:
            questions.append(element)

print(questions)

['A natural disaster occurs in Manhattan. What impact does this have on markets?', 'Add up all the numbers between 0-100.', 'Are there any particular companies that interest you?', 'Are there aspects to your job that are repetitious?', 'Are you a Coronation Street or EastEnders fan?', 'Are you a goal-oriented person?', 'Are you a leader or a follower?', 'Are you a lone wolf?', 'Are you a team player?', 'Are you better with data or people?', 'Are you comfortable with a lack of deadlines?', 'Are you comfortable with strict deadlines?', 'Are you currently taking or enrolled to take any job related educational courses?', 'Are you currently under any employment contract obligation form current or previous employers?', 'Are you good at accepting delegation?', 'Are you good at delegating tasks?', 'Are you hoping for advancement within the company?', 'Are you looking for a permanent or temporary position at the company?', 'Are you the type of student for whom conducting independent research ha

In [129]:
#We will iterate over all the saved questions and translate them all, we have 1003
print(len(questions))
print(len(allQuestions))

982
1003


In [130]:
print(questions)

['A natural disaster occurs in Manhattan. What impact does this have on markets?', 'Add up all the numbers between 0-100.', 'Are there any particular companies that interest you?', 'Are there aspects to your job that are repetitious?', 'Are you a Coronation Street or EastEnders fan?', 'Are you a goal-oriented person?', 'Are you a leader or a follower?', 'Are you a lone wolf?', 'Are you a team player?', 'Are you better with data or people?', 'Are you comfortable with a lack of deadlines?', 'Are you comfortable with strict deadlines?', 'Are you currently taking or enrolled to take any job related educational courses?', 'Are you currently under any employment contract obligation form current or previous employers?', 'Are you good at accepting delegation?', 'Are you good at delegating tasks?', 'Are you hoping for advancement within the company?', 'Are you looking for a permanent or temporary position at the company?', 'Are you the type of student for whom conducting independent research ha

In [131]:
#Save all the questions in a CSV
df = pd.DataFrame({'Interviewer Questions':questions}) 
df.to_csv('Interviewer_Questions.csv', index=False, encoding='utf-8')

In [132]:
from selenium.webdriver.common.by import By
import pyperclip

#Now lets go to the google traductor page
driver.get('https://translate.google.es/?hl=es&sl=en&tl=es&op=translate');
time.sleep(5) # Let the user actually see something!

In [133]:
#Accept google cookies
driver.find_element(By.XPATH, "//span[normalize-space()='Acepto']").click()

In [134]:
# Global list to save the translated questions
questions_translated = []

In [135]:
len(questions)

982

In [137]:
i=0
stop=0

# Get the search box and clean it 
search_box = driver.find_element(By.CLASS_NAME, "er8xn")
search_box.clear()

def questions_traductor(index):
    i=index
    last_answer=[]
    translation=[]
    
    while i < len(questions):
        #if the program fails, stop will save the index and it will restart at the point where failed
        stop=i
        
        # Print the position and question to see if everything goes well
        print(i)
        print(questions[i])        
        
        # If the question actually exists (errors or blank lines could be imported in the scrapping)
        if questions[i] is not None:

            # Get the search_box to send the question
            try:
                search_box = driver.find_element(By.CLASS_NAME, "er8xn")
            except:
                questions_traductor(stop)

            # Questions sended to the translator
            search_box.send_keys(questions[i])
            
            # Counter
            iteration=0
            
            # Sleep needed to let the traductor respond, if the actual question and the last one are the same
            # we must keep waiting, this solution is better than waiting a constant amount of time 
            while (last_answer==translation):
                
                #if the program is stuck in here (>5 iterations) its probably because the 2 original sentences are similar
                #and only change a comma or any other punctuaction mark, but that is ignored by the translator because
                #the destiny language doesnt need it so the program detects that 2 very similar sentences are the same
                #and keeps trying to make a better traduction
                iteration=iteration+1
                if iteration>5:
                    break
                
                print("Equals, more time needed for the translator to respond")
                time.sleep(1)
                try:
                    # find the copy translation button and click on it
                    driver.find_element(By.XPATH, "//button[@aria-label='Copiar traducción']//div[@class='VfPpkd-Bz112c-RLmnJb']").click()
                    translation = []
                    translation=pyperclip.paste()
                except:
                    questions_traductor(stop)
            
            print("Last question", last_answer)
            print("Current question", translation)

            # Append the translation
            questions_translated.append(translation)

            # Clear the search box to keep translating
            search_box.clear()
            
            # The actual answer will be the last in the next iteration so we can check and wait till traductor responds
            last_answer=translation
            last=questions[i]
            
            
        
        # Next question, doesnt matter if the question before existed or not
        i=i+1

In [138]:
questions_traductor(stop)

0
A natural disaster occurs in Manhattan. What impact does this have on markets?
ultima pregunta 1 []
pregunta actual 1 []
Equals, more time needed for the translator to respond
ultima pregunta 2 []
pregunta actual 2 Se produce un desastre natural en Manhattan. ¿Qué impacto tiene esto en los mercados?
Se produce un desastre natural en Manhattan. ¿Qué impacto tiene esto en los mercados?
1
Add up all the numbers between 0-100.
ultima pregunta 1 Se produce un desastre natural en Manhattan. ¿Qué impacto tiene esto en los mercados?
pregunta actual 1 Se produce un desastre natural en Manhattan. ¿Qué impacto tiene esto en los mercados?
Equals, more time needed for the translator to respond
ultima pregunta 2 Se produce un desastre natural en Manhattan. ¿Qué impacto tiene esto en los mercados?
pregunta actual 2 Suma todos los números entre 0-100.
Suma todos los números entre 0-100.
2
Are there any particular companies that interest you?
ultima pregunta 1 Suma todos los números entre 0-100.
preg

ultima pregunta 1 ¿Eres el tipo de estudiante para quien la realización de investigaciones independientes ha sido una experiencia positiva?
pregunta actual 1 ¿Eres el tipo de estudiante para quien la realización de investigaciones independientes ha sido una experiencia positiva?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Eres el tipo de estudiante para quien la realización de investigaciones independientes ha sido una experiencia positiva?
pregunta actual 2 ¿Estás dispuesto a hacer sacrificios para esta empresa?
¿Estás dispuesto a hacer sacrificios para esta empresa?
20
Are you willing to relocate?
ultima pregunta 1 ¿Estás dispuesto a hacer sacrificios para esta empresa?
pregunta actual 1 ¿Estás dispuesto a hacer sacrificios para esta empresa?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Estás dispuesto a hacer sacrificios para esta empresa?
pregunta actual 2 ¿Estás dispuesto a mudarte?
¿Estás dispuesto a mudarte?
21
Are you w

ultima pregunta 1 ¿Me puede dar un ejemplo de su creatividad / gestión / habilidades organizativas?
pregunta actual 1 ¿Me puede dar un ejemplo de su creatividad / gestión / habilidades organizativas?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Me puede dar un ejemplo de su creatividad / gestión / habilidades organizativas?
pregunta actual 2 ¿Puedes hacer un mercado basado en el valor esperado de un rollo de dados?
¿Puedes hacer un mercado basado en el valor esperado de un rollo de dados?
37
Can you perform Internet research? Please describe to me your steps in doing so.
ultima pregunta 1 ¿Puedes hacer un mercado basado en el valor esperado de un rollo de dados?
pregunta actual 1 ¿Puedes hacer un mercado basado en el valor esperado de un rollo de dados?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Puedes hacer un mercado basado en el valor esperado de un rollo de dados?
pregunta actual 2 ¿Puedes realizar investigaciones de Inter

ultima pregunta 2 ¿Puedes pensar en un tiempo una idea de la tuya fue rechazada? Cuéntame sobre eso.
pregunta actual 2 ¿Puedes trabajar sin supervisión?
¿Puedes trabajar sin supervisión?
48
Could you have done better in your last job?
ultima pregunta 1 ¿Puedes trabajar sin supervisión?
pregunta actual 1 ¿Puedes trabajar sin supervisión?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Puedes trabajar sin supervisión?
pregunta actual 2 ¿Podrías haberlo hecho mejor en tu último trabajo?
¿Podrías haberlo hecho mejor en tu último trabajo?
49
Define commitment.
ultima pregunta 1 ¿Podrías haberlo hecho mejor en tu último trabajo?
pregunta actual 1 ¿Podrías haberlo hecho mejor en tu último trabajo?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Podrías haberlo hecho mejor en tu último trabajo?
pregunta actual 2 Definir el compromiso.
Definir el compromiso.
50
Define cooperation.
ultima pregunta 1 Definir el compromiso.
pregunta actual 1 Defi

ultima pregunta 1 Describa un cambio importante que ocurrió en un trabajo que realizó. ¿Cómo se adaptó a este cambio?
pregunta actual 1 Describa un cambio importante que ocurrió en un trabajo que realizó. ¿Cómo se adaptó a este cambio?
Equals, more time needed for the translator to respond
ultima pregunta 2 Describa un cambio importante que ocurrió en un trabajo que realizó. ¿Cómo se adaptó a este cambio?
pregunta actual 2 Describa un proyecto que requirió una gran cantidad de energía durante un período prolongado de tiempo.
Describa un proyecto que requirió una gran cantidad de energía durante un período prolongado de tiempo.
68
Describe a recent news article that interested you.
ultima pregunta 1 Describa un proyecto que requirió una gran cantidad de energía durante un período prolongado de tiempo.
pregunta actual 1 Describa un proyecto que requirió una gran cantidad de energía durante un período prolongado de tiempo.
Equals, more time needed for the translator to respond
ultima preg

...

ultima pregunta 1 ¿Por qué cambiaste los mayores?
pregunta actual 1 ¿Por qué cambiaste los mayores?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Por qué cambiaste los mayores?
pregunta actual 2 ¿Por qué escogiste esta carrera?
¿Por qué escogiste esta carrera?
925
Why did you choose to attend your college?
ultima pregunta 1 ¿Por qué escogiste esta carrera?
pregunta actual 1 ¿Por qué escogiste esta carrera?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Por qué escogiste esta carrera?
pregunta actual 2 ¿Por qué elegiste asistir a tu universidad?
¿Por qué elegiste asistir a tu universidad?
926
Why did you choose your degree subject?
ultima pregunta 1 ¿Por qué elegiste asistir a tu universidad?
pregunta actual 1 ¿Por qué elegiste asistir a tu universidad?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Por qué elegiste asistir a tu universidad?
pregunta actual 2 ¿Por qué elegiste su asunto de grado?
¿Por qué 

ultima pregunta 2 ¿Por qué debo contratarte?
pregunta actual 2 Porque deberíamos contratarte?
Porque deberíamos contratarte?
945
Why was there a gap in your employment between [insert dates] and [insert date]?
ultima pregunta 1 Porque deberíamos contratarte?
pregunta actual 1 Porque deberíamos contratarte?
Equals, more time needed for the translator to respond
ultima pregunta 2 Porque deberíamos contratarte?
pregunta actual 2 ¿Por qué hubo una brecha en su empleo entre [Fechas de inserción] y [Fecha de inserción]?
¿Por qué hubo una brecha en su empleo entre [Fechas de inserción] y [Fecha de inserción]?
946
Why were you transferred/promoted?
ultima pregunta 1 ¿Por qué hubo una brecha en su empleo entre [Fechas de inserción] y [Fecha de inserción]?
pregunta actual 1 ¿Por qué hubo una brecha en su empleo entre [Fechas de inserción] y [Fecha de inserción]?
Equals, more time needed for the translator to respond
ultima pregunta 2 ¿Por qué hubo una brecha en su empleo entre [Fechas de inserci

ultima pregunta 2 Escríbanos sobre un incidente en tu vida que muestra cómo se enfrentó a una dificultad / desafío y cómo lo manejó.
pregunta actual 2 Estás en una fiesta con un amigo y 10 personas están presentes, incluido usted y el amigo. Tu amigo te convierte en una apuesta que para cada persona que encuentres que tiene el mismo cumpleaños que tú, obtienes $ 1; Para cada persona que encuentra que no tiene el mismo cumpleaños que tú, obtiene $ 2. ¿Aceptarías la apuesta?
Estás en una fiesta con un amigo y 10 personas están presentes, incluido usted y el amigo. Tu amigo te convierte en una apuesta que para cada persona que encuentres que tiene el mismo cumpleaños que tú, obtienes $ 1; Para cada persona que encuentra que no tiene el mismo cumpleaños que tú, obtiene $ 2. ¿Aceptarías la apuesta?
962
You are given 2 eggs. You have access to a 100-story building. Eggs can be very hard or very fragile means it may break if dropped from the first floor or may not even break if dropped from 1

ultima pregunta 2 Tienes ocho bolas de todo el mismo tamaño. 7 de ellos pesan lo mismo, y uno de ellos pesa un poco más. ¿Cómo puedes encontrar la pelota que es más pesada usando un equilibrio y solo pesa las bolas dos veces?
pregunta actual 2 Tienes cinco piratas, clasificados de 5 a 1 en orden descendente. El pirata superior tiene el derecho de proponer cómo se deben dividir 100 monedas de oro entre ellos. Pero los otros votan en su plan, y si menos de la mitad están de acuerdo con él, se mata. ¿Cómo debería asignar el oro para maximizar su parte, pero vivir para disfrutarlo? (Pista: un pirata termina con el 98 por ciento del oro.)
Tienes cinco piratas, clasificados de 5 a 1 en orden descendente. El pirata superior tiene el derecho de proponer cómo se deben dividir 100 monedas de oro entre ellos. Pero los otros votan en su plan, y si menos de la mitad están de acuerdo con él, se mata. ¿Cómo debería asignar el oro para maximizar su parte, pero vivir para disfrutarlo? (Pista: un pirata

ultima pregunta 2 Usted está en medio de una tarea urgente y alguien más senior exige alguna información instantánea. ¿A qué te dedicas?
pregunta actual 2 Eres el CEO de una empresa. ¿Prefieres elevar la deuda o la equidad? ¿Por qué?
Eres el CEO de una empresa. ¿Prefieres elevar la deuda o la equidad? ¿Por qué?
976
Your boss assigns you a project and then goes on holiday. What do you do?
ultima pregunta 1 Eres el CEO de una empresa. ¿Prefieres elevar la deuda o la equidad? ¿Por qué?
pregunta actual 1 Eres el CEO de una empresa. ¿Prefieres elevar la deuda o la equidad? ¿Por qué?
Equals, more time needed for the translator to respond
ultima pregunta 2 Eres el CEO de una empresa. ¿Prefieres elevar la deuda o la equidad? ¿Por qué?
pregunta actual 2 Su jefe le asigna un proyecto y luego se va de vacaciones. ¿A qué te dedicas?
Su jefe le asigna un proyecto y luego se va de vacaciones. ¿A qué te dedicas?
977
Your resume suggests that you may be over-qualified or too experienced for this posit

In [144]:
#Save all the questions in a CSV
df = pd.DataFrame({'Interviewer Questions Translated':questions_translated}) 
df.to_csv('Interviewer_Questions_Translated_v2.csv', index=False, encoding='utf-8')

In [140]:
# Check that the lenght is the same as the original questions list
len(questions_translated)

982

In [141]:
print(questions_translated)

['Se produce un desastre natural en Manhattan. ¿Qué impacto tiene esto en los mercados?', 'Suma todos los números entre 0-100.', '¿Hay alguna empresa en particular que te interese?', '¿Hay aspectos de su trabajo que son repetitivos?', '¿Eres una calle de coronación o fanático de Eastenders?', '¿Eres una persona orientada a objetivos?', '¿Eres un líder o un seguidor?', '¿Eres un lobo solitario?', '¿Eres un jugador de equipo?', '¿Eres mejor con datos o personas?', '¿Estás cómodo con la falta de plazos?', '¿Estás cómodo con plazos estrictos?', '¿Actualmente está tomando o inscrito para tomar los cursos educativos relacionados con el trabajo?', '¿Está actualmente bajo cualquier forma de obligación de contrato de empleo, empleadores actuales o anteriores?', '¿Eres bueno aceptando la delegación?', '¿Eres bueno en la delegación de tareas?', '¿Está esperando un avance dentro de la empresa?', '¿Está buscando una posición permanente o temporal en la empresa?', '¿Eres el tipo de estudiante para q

In [142]:
#Loop to check if there is any repeated question or none

last=[]

for element in questions_translated:
    if element != None :
        if element not in last:
            last.append(element)
        else:
            print("REPEATED")
    else:
        print("NONE")

REPETIDO
REPETIDO


In [143]:
#Close the driver
driver.quit()